### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@datalakete.dfs.core.windows.net/orders")

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))\
       .withColumn("year", year(col("order_date")))\
       .drop('_rescued_data')
df.display()

In [0]:
df = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df.display()

In [0]:
df = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df.display()

In [0]:
df = df.withColumn("rno_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df.display()

### **Classes OOPs**

In [0]:
class Windows:

  def dense_rank(self,df):
    df_dence_rank = df.withColumn("dense_rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_dence_rank
  
  def rank(self,df):
    df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_rank
  
  def rno(self,df):
    df_row_number = df.withColumn("row_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_row_number
  


In [0]:
obj = Windows()

obj.dense_rank(df).display()

### **Silver: Data Writing in delta**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@datalakete.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_av_cata.silver.orders
USING DELTA
LOCATION "abfss://silver@datalakete.dfs.core.windows.net/orders"

In [0]:
%sql
select * from databricks_av_cata.silver.orders